# Read the Code of Renormalizer

`Op` initializes the operator from
- the representation of 
- the Degree of Freedom (DoF) (the indices of the operator)
- prefactor


In [1]:
from renormalizer.model import Op


x = Op("X", 0, 0.5)
y = Op("Y", 1, 0.2)

print(dir(x))
print(x * (x + y))

2023-02-22 15:15:17,228[INFO] Use NumPy as backend
2023-02-22 15:15:17,230[INFO] use 64 bits


['__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_factor', 'dofs', 'factor', 'identity', 'is_identity', 'product', 'qn', 'qn_list', 'same_term', 'split_elementary', 'split_symbol', 'squeeze_identity', 'symbol', 'to_tuple']
[Op('X X', [0, 0], 0.25), Op('X Y', [0, 1], 0.1)]


In [2]:
from renormalizer import Model, Op, BasisHalfSpin,  Mps, Mpo, optimize_mps

# define the # of spins
nspin = 32

# define the model

ham_terms = []
for ispin in range(nspin-1):
    op1 = Op("sigma_z sigma_z", [ispin, ispin+1], 1.0/4)
    op2 = Op("sigma_+ sigma_-", [ispin, ispin+1], 1.0/2)
    op3 = Op("sigma_- sigma_+", [ispin, ispin+1], 1.0/2)
    ham_terms.extend([op1, op2, op3])

# set the spin order and local basis
basis = [BasisHalfSpin(i) for i in range(nspin)]
# construct Hamiltonian MPO
model = Model(basis, ham_terms)
mpo = Mpo(model)
print(f"mpo_bond_dims:{mpo.bond_dims}")

2023-02-22 15:15:21,505[DEBUG] # of operator terms: 93
2023-02-22 15:15:21,506[DEBUG] symbolic mpo algorithm: Hopcroft-Karp
2023-02-22 15:15:21,508[DEBUG] Input operator terms: 93
2023-02-22 15:15:21,533[DEBUG] After combination of the same terms: 93


mpo_bond_dims:[1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1]


In [14]:
from renormalizer import BasisMultiElectronVac

b = BasisMultiElectronVac(4)

b.op_mat("a")

TypeError: object of type 'int' has no len()

In [19]:
import logging
import time

import numpy as np

from renormalizer import Model, Mps, Mpo, optimize_mps
from renormalizer.model import h_qc
from renormalizer.utils import log

'''
water sto-3g (10e,7o)
O     0.0000000    0.0000000   -0.0644484,
H     0.7499151    0.0000000    0.5114913,
H    -0.7499151    0.0000000    0.5114913,
'''

start = time.time()
dump_dir = "./"
job_name = "qc"  #########
log.set_stream_level(logging.DEBUG)
log.register_file_output(dump_dir+job_name+".log", mode="w")
logger = logging.getLogger("renormalizer")

spatial_norbs = 7
spin_norbs = spatial_norbs * 2
h1e, h2e, nuc = h_qc.read_fcidump("h2o_fcidump.txt", spatial_norbs)

# Potential for H2O has high symmetry and constructed MPO is smaller
# than MPO in normal case. Use random potential to compare with normal MPO.
RANDOM_INTEGRAL = False
if RANDOM_INTEGRAL:
    h1e = np.random.uniform(-1,1,size=(spin_norbs,spin_norbs))
    h2e = np.random.uniform(-1,1,size=(spin_norbs,spin_norbs,spin_norbs,spin_norbs))
    h1e = 0.5*(h1e+h1e.T)
    h2e = 0.5*(h2e+h2e.transpose((2,3,0,1)))

basis, ham_terms = h_qc.qc_model(h1e, h2e)
print(basis[0].op_mat("X"))

2023-02-19 15:09:36,530[INFO] nuclear repulsion: 9.307155269556182
2023-02-19 15:09:36,536[INFO] spin norbs: 14


[[0. 1.]
 [1. 0.]]


In [3]:
import logging

from renormalizer.sbm import SpinBosonDynamics, param2mollist, SpectralDensityFunction
from renormalizer.model import Phonon
from renormalizer.model.basis import BasisSHO
from renormalizer.utils import Quantity, CompressConfig, EvolveConfig, log

log.init_log(logging.INFO)

alpha = 0.5
raw_delta = Quantity(1)
raw_omega_c = Quantity(40)
n_phonons = 20
renormalization_p = 1
sdf = SpectralDensityFunction(alpha, raw_omega_c)

delta, max_omega = sdf.adiabatic_renormalization(raw_delta, renormalization_p)
omega_list, displacement_list = sdf.trapz(n_phonons, 0.0, max_omega.as_au())

ph_list = [Phonon.simplest_phonon(o, d) for o,d in zip(omega_list, 
displacement_list)]

basis = []

for iph, ph in enumerate(ph_list):
            basis.append(BasisSHO(iph, ph.omega[0], ph.n_phys_dim))
            
print([ph.n_phys_dim for ph in ph_list])
print([ph.omega for ph in ph_list])
print([ph.dis for ph in ph_list])
lastbasis = basis[-1]


[4, 4, 4, 4, 4, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 16]
[[0.04336565713446032, 0.04336565713446032], [0.041141777281411065, 0.041141777281411065], [0.038917897428361825, 0.038917897428361825], [0.03669401757531257, 0.03669401757531257], [0.03447013772226333, 0.03447013772226333], [0.032246257869214076, 0.032246257869214076], [0.030022378016164832, 0.030022378016164832], [0.027798498163115585, 0.027798498163115585], [0.02557461831006634, 0.02557461831006634], [0.023350738457017094, 0.023350738457017094], [0.021126858603967846, 0.021126858603967846], [0.0189029787509186, 0.0189029787509186], [0.01667909889786935, 0.01667909889786935], [0.014455219044820104, 0.014455219044820104], [0.012231339191770857, 0.012231339191770857], [0.010007459338721611, 0.010007459338721611], [0.007783579485672364, 0.007783579485672364], [0.005559699632623117, 0.005559699632623117], [0.00333581977957387, 0.00333581977957387], [0.0011119399265246235, 0.0011119399265246235]]
[[0.0, 0.7685270491576383], [0.

In [7]:
print(basis[0].op_mat("b^\dagger"))
print(basis[0].op_mat("b"))
print(basis[0].op_mat("b^\dagger b"))

[[0.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [0.         1.41421356 0.         0.        ]
 [0.         0.         1.73205081 0.        ]]
[[0.         1.         0.         0.        ]
 [0.         0.         1.41421356 0.        ]
 [0.         0.         0.         1.73205081]
 [0.         0.         0.         0.        ]]
[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 3.]]
